In [1]:
from config import api_key
import requests
import csv
import pandas as pd
import pprint
import json
import time 

In [9]:
# Get data from https://rapidapi.com/
# title/get-ratings

# load csv file
url = "https://imdb8.p.rapidapi.com/title/get-ratings"

titles = pd.read_csv("./Resources/IMDB_API_adventure100.csv")
ratings = []



In [12]:
for title in titles['title']:
	time.sleep(0.5)
	querystring = {"tconst":title}
	headers = {
		'x-rapidapi-key': api_key,
		'x-rapidapi-host': "imdb8.p.rapidapi.com"
		}
	response = requests.request("GET", url, headers=headers, params=querystring)

	pprint.pprint(response.json())

	
	ratinginfo={}
	ratinginfo['id'] = title
		
	if 'canRate' not in response.json():
		print(f"No rating for {title}")
		ratinginfo['canRate'] = False
	else:
		ratinginfo['canRate'] = response.json()['canRate']
		if response.json()['canRate'] == True:
			ratinginfo['rating'] = response.json()['rating']
			ratinginfo['ratingCount']=response.json()['ratingCount']
			ratinginfo['ratingsHistograms'] = response.json()['ratingsHistograms']
			
			if 'topRank' not in response.json():
				print(f"No toprating for {title}")
			else:
				ratinginfo['topRank'] = response.json()['topRank']
				ratinginfo['bottomRank']=response.json()['bottomRank']

		
		ratings.append(ratinginfo)


{'@type': 'imdb.api.title.ratings',
 'bottomRank': 7003,
 'canRate': True,
 'id': '/title/tt11858890/',
 'rating': 7.2,
 'ratingCount': 35237,
 'ratingsHistograms': {'Aged 18-29': {'aggregateRating': 7.2,
                                      'demographic': 'Aged 18-29',
                                      'histogram': {'1': 25,
                                                    '10': 376,
                                                    '2': 20,
                                                    '3': 29,
                                                    '4': 109,
                                                    '5': 227,
                                                    '6': 651,
                                                    '7': 1225,
                                                    '8': 1187,
                                                    '9': 530},
                                      'totalRatings': 4379},
                       'Aged 30-44': {'aggrega

In [15]:
df=pd.DataFrame(ratings)
# reorder columns
df=df[['id','canRate','rating','ratingCount','topRank','bottomRank','ratingsHistograms']]
#unqiue values
df=df.drop_duplicates(subset=['id'])
df.to_csv("./Resources/IMDB_API_adventure100_rating.csv", index=False, header=True)
df.to_json("./Resources/IMDB_API_adventure100_rating.json", orient="records")